In [1]:
%ls

db.sqlite3         Gendata.ipynb  manage.py*  myproject/    Quiz Django.ipynb
Exam Django.ipynb  init.sh        myapp/      notebook.sh*  Quiz panda.ipynb


In [2]:
import sqlite3
import pandas as pd
con = sqlite3.connect('db.sqlite3')
df = pd.read_sql_query("SELECT * FROM sqlite_master", con)
df

,type,name,tbl_name,rootpage,sql
0,table,django_migrations,django_migrations,2,"CREATE TABLE ""django_migrations"" (""id"" integer..."
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,auth_group,auth_group,9,"CREATE TABLE ""auth_group"" (""id"" integer NOT NU..."
3,index,sqlite_autoindex_auth_group_1,auth_group,10,None
4,table,auth_group_permissions,auth_group_permissions,11,"CREATE TABLE ""auth_group_permissions"" (""id"" in..."
5,table,auth_user_groups,auth_user_groups,15,"CREATE TABLE ""auth_user_groups"" (""id"" integer ..."
6,table,auth_user_user_permissions,auth_user_user_permissions,16,"CREATE TABLE ""auth_user_user_permissions"" (""id..."
7,index,auth_group_permissions_group_id_permission_id_...,auth_group_permissions,20,CREATE UNIQUE INDEX auth_group_permissions_gro...
8,index,auth_group_permissions_group_id_b120cbf9,auth_group_permissions,21,"CREATE INDEX ""auth_group_permissions_group_id_..."
9,index,auth_group_permissions_permission_id_84c5c92e,auth_group_permissions,23,"CREATE INDEX ""auth_group_permissions_permissio..."


In [3]:
df_member = pd.read_sql_query("SELECT * FROM myapp_member", con, index_col='id')
df_room = pd.read_sql_query("SELECT * FROM myapp_room", con, index_col='id')
df_rent = pd.read_sql_query("SELECT * FROM myapp_rent", con, index_col='id', parse_dates=['start','stop'])

In [6]:
df=df_rent.merge(df_member,left_on='member_id',right_index=True)\
    .merge(df_room, left_on='room_id',right_index=True)
df['hours']=(df_rent.stop-df_rent.start).apply(lambda x: x.seconds)//3600
df['cost']=df['hour_price']*df['hours']
df.groupby(by=['member_id','name'])\
    .agg({'cost' : ['sum']})\
    .sort_values(by=[('cost','sum')], ascending=False)

,,cost
,,sum
member_id,name,
94,Oscar Smith,15116.0
59,Jacob Jones,14730.0
89,George Davies,14303.0
68,Harry Williams,13910.0
77,Jack Jones,13668.0
82,Claire Brown,13593.0
10,Harry Jones,13266.0
8,Thomas Davies,13043.0


In [23]:
from datetime import datetime
start = datetime.strptime( "2018-06-01T00:00:00", "%Y-%m-%dT%H:%M:%S" )
stop = datetime.strptime( "2018-06-30T23:59:59", "%Y-%m-%dT%H:%M:%S" )
df=df_rent
df=df[(start<=df.start) & (df.stop<=stop)]
df=df.merge(df_member,left_on='member_id',right_index=True)\
    .merge(df_room, left_on='room_id',right_index=True)
df['hours']=(df_rent.stop-df_rent.start).apply(lambda x: x.seconds)//3600
df['cost']=df['hour_price']*df['hours']
df.groupby(by=['member_id','name'])\
    .agg({'cost' : ['sum']})\
    .sort_values(by=[('cost','sum')], ascending=False)
df.sort_values(by=['hours'])

,start,stop,member_id,room_id,cost,name,mobile,number,hour_price,isavailable,hours
id,,,,,,,,,,,
1585,2018-06-30 04:01:18.462109,2018-06-30 05:01:18.462109,11,6,105.0,William Evans,,101,105,1,1.0
1438,2018-06-23 08:01:18.462109,2018-06-23 09:01:18.462109,23,9,130.0,Oscar Johnson,,104,130,1,1.0
1514,2018-06-27 09:01:18.462109,2018-06-27 10:01:18.462109,77,9,130.0,Jack Jones,,104,130,1,1.0
1319,2018-06-18 10:01:18.462109,2018-06-18 11:01:18.462109,61,10,199.0,Claire Evans,,105,199,1,1.0
1317,2018-06-18 09:01:18.462109,2018-06-18 10:01:18.462109,63,9,130.0,Harry Evans,,104,130,1,1.0
1084,2018-06-08 10:01:18.462109,2018-06-08 11:01:18.462109,63,9,130.0,Harry Evans,,104,130,1,1.0
1495,2018-06-26 10:01:18.462109,2018-06-26 11:01:18.462109,20,6,105.0,Oscar Evans,,101,105,1,1.0
1587,2018-06-30 04:01:18.462109,2018-06-30 05:01:18.462109,31,9,130.0,Oscar Wilson,,104,130,1,1.0
1067,2018-06-07 19:01:18.462109,2018-06-07 20:01:18.462109,31,9,130.0,Oscar Wilson,,104,130,1,1.0
